## Importing the libraries 

In [1]:
# Importing required libraries
import pandas as pd
import numpy as np
from numpy.random import seed
seed(123)
import matplotlib.pyplot as plt
%matplotlib inline
from datetime import datetime
import seaborn as sns
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import explained_variance_score, mean_squared_error, r2_score
import time
from IPython.display import SVG
from statsmodels.tsa.seasonal import seasonal_decompose
import geopandas as gpd
from keras import models, layers, optimizers, regularizers
from keras.utils.vis_utils import model_to_dot
from sklearn.feature_extraction.text import CountVectorizer

## Importing the data

In [3]:
%%time
#! rm -r data.insideairbnb.com
# DC data:
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-12-22/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-11-22/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-10-18/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-09-22/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-08-22/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-07-15/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-06-16/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-05-20/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-04-15/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-03-12/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-02-11/data/listings.csv.gz
! wget -xq http://data.insideairbnb.com/united-states/dc/washington-dc/2019-01-17/data/listings.csv.gz

! du -sh data.insideairbnb.com

Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
Illegal variable name.
185M	data.insideairbnb.com
CPU times: user 128 ms, sys: 119 ms, total: 247 ms
Wall time: 7.48 s


### Data Cleaning 

In [2]:
def bin_column(df, col, bins, labels, na_label='unknown'):
    """
    Takes in a column name, bin cut points and labels, replaces the original column with a
    binned version, and replaces nulls (with 'unknown' if unspecified).
    """   
    df[col] = pd.cut(df[col], bins=bins, labels=labels, include_lowest=True)
    df[col] = df[col].astype('str')
    df[col] = df[col].str.replace("nan",na_label)
#    df[col].fillna(na_label, inplace=True)

In [3]:
def DropColumns(df):
    
    df = df.replace({'f': 0, 't': 1})
    # Initail columns to drop
    cols1 = ['listing_url', 'scrape_id', 'last_scraped', 'name', 'summary', 'space', 'description', 'neighborhood_overview', 'notes', 'transit', 'access', 'interaction', 'house_rules', 'thumbnail_url', 'medium_url', 'picture_url', 'xl_picture_url', 'host_url', 'host_name', 'host_location', 'host_about', 'host_thumbnail_url', 'host_picture_url', 'host_neighbourhood', 'host_verifications', 'calendar_last_scraped']

    # These are columns with majority of null values
    cols2 = ['experiences_offered', 'host_acceptance_rate', 'neighbourhood_group_cleansed', 'square_feet', 'weekly_price', 'monthly_price', 'license', 'jurisdiction_names']

    # host_listings_count and host_total_listings_count are almost the same. Therefore one of these columns can be dropped.   
    cols3 = ['host_total_listings_count', 'calculated_host_listings_count', 'calculated_host_listings_count_entire_homes', 'calculated_host_listings_count_private_rooms', 'calculated_host_listings_count_shared_rooms']

    #Latitude and longitude will be saved in a dataframe for later use (see modelling section below).
    cols4 = ['latitude', 'longitude']
    
    # There are multiple columns for minimum and maximum night stays
    cols5 = ['minimum_minimum_nights', 'maximum_minimum_nights', 'minimum_maximum_nights', 'maximum_maximum_nights', 'minimum_nights_avg_ntm', 'maximum_nights_avg_ntm']
  
    #These will be highly correlated with number_of_reviews and so will be dropped.
    cols6 = ['number_of_reviews_ltm']

    # There are multiple columns for property location, we only keep 'neighboorhood_cleansed'
    cols7 = ['zipcode', 'street', 'city', 'state','market','smart_location', 'neighbourhood', 'country_code', 'country']
    
    # These columns has value of 1 for all listing 
    cols8 = ['has_availability', 'host_has_profile_pic', 'is_business_travel_ready', 'require_guest_phone_verification', 'require_guest_profile_picture', 'requires_license']
    
    cols = cols1 + cols2 + cols3 + cols4 + cols5 + cols6 + cols7 + cols8
    return df.drop(cols, axis=1)


def cleanpropertytype(df):
    # Replacing categories that are types of houses or apartments
    df.property_type.replace({
        'Townhouse': 'House',
        'Serviced apartment': 'Apartment',
        'Loft': 'Apartment',
        'Bungalow': 'House',
        'Cottage': 'House',
        'Villa': 'House',
        'Tiny house': 'House',
        'Earth house': 'House',
        'Chalet': 'House'  
        }, inplace=True)

    # Replacing other categories with 'other'
    df.loc[~df.property_type.isin(['House', 'Apartment']), 'property_type'] = 'Other'
    return df

def cleanrooms(df):
    for col in ['bedrooms', 'beds','bathrooms']:
        df[col].fillna(df[col].median(), inplace=True)
    return df

def cleanprice(df):
    df.price = df.price.str[1:-3]
    df.price = df.price.str.replace(",", "")
    df.price = df.price.astype('int64')
    return df

def cleansecuritydeposit(df):
    df.security_deposit = df.security_deposit.str[1:-3]
    df.security_deposit = df.security_deposit.str.replace(",", "")
    df.security_deposit.fillna(0, inplace=True)
    df.security_deposit = df.security_deposit.astype('int64')
    return df

def cleancleaningfee(df):
    df.cleaning_fee = df.cleaning_fee.str[1:-3]
    df.cleaning_fee = df.cleaning_fee.str.replace(",", "")
    df.cleaning_fee.fillna(0, inplace=True)
    df.cleaning_fee = df.cleaning_fee.astype('int64')
    return df 

def cleanextrapeople(df):
    df.extra_people = df.extra_people.str[1:-3]
    df.extra_people = df.extra_people.str.replace(",", "")
    df.extra_people.fillna(0, inplace=True)   # using $0 to replace the missing value
    df.extra_people = df.extra_people.astype('int64')
    return df 

def cleanhostresponsetime(df):
    df.host_response_time.fillna("unknown", inplace=True)
    return df

def cleanhostresponserate(df):
    # Removing the % sign from the host_response_rate string and converting to an integer
    df.host_response_rate = df.host_response_rate.str[:-1].astype('float64')
    
    # Bin into four categories
    df.host_response_rate = pd.cut(df.host_response_rate, bins=[0, 50, 90, 99, 100], labels=['0-49%', '50-89%', '90-99%', '100%'], include_lowest=True)

    # Converting to string
    df.host_response_rate = df.host_response_rate.astype('str')

    # Replace nulls with 'unknown'
    df.host_response_rate.replace('nan', 'unknown', inplace=True)

    # Category counts
    df.host_response_rate.value_counts()
    return df

def cleancancancellationpolicy(df):
    # Replacing categories
    df.cancellation_policy.replace({
        'super_strict_30': 'strict_14_with_grace_period',
        'super_strict_60': 'strict_14_with_grace_period',
        'strict': 'strict_14_with_grace_period',
        'luxury_moderate': 'moderate'
        }, inplace=True)
    return df

def clean(df, year, month, day):
    df = cleanpropertytype(df)
    df = cleanrooms(df)
    df = cleanprice(df)
    df = cleansecuritydeposit(df)
    df = cleancleaningfee(df)
    df = cleanhostresponsetime(df)
    df = cleanhostresponserate(df)
    df = cleanextrapeople(df)
    df = cleancancancellationpolicy(df)
    df = cleanhostsince(df, year, month, day)
    df = cleanfirstreview(df, year, month, day)
    df = cleanlastreview(df, year, month, day)
    return df 

def cleanhostsince(df, year, month, day):
    # Converting to datetime
    df.host_since = pd.to_datetime(df.host_since) 

    # Calculating the number of days
    df['host_days_active'] = (datetime(year, month, day) - df.host_since).astype('timedelta64[D]')
    df.host_days_active = df.host_days_active.fillna(df.host_days_active.median())
    return df

def cleanfirstreview(df, year, month, day):
    df.first_review = pd.to_datetime(df.first_review) # Converting to datetime
    df['time_since_first_review'] = (datetime(year, month, day) - df.first_review).astype('timedelta64[D]')
    bin_column(df, 'time_since_first_review',
           bins=[0, 182, 365, 730, 1460, max(df.time_since_first_review)],
           labels=['0-6 months','6-12 months','1-2 years','2-3 years','4+ years'],
           na_label='no reviews')
    return df

def cleanlastreview(df, year, month, day):
    df.last_review = pd.to_datetime(df.last_review) # Converting to datetime
    df['time_since_last_review'] = (datetime(year, month, day) - df.last_review).astype('timedelta64[D]')
    bin_column(df_jan, 'time_since_last_review',
           bins=[0, 182, 365, 730, 1460, max(df_jan.time_since_last_review)],
           labels=['0-6 months','6-12 months','1-2 years','2-3 years','4+ years'],
           na_label='no reviews')
    return df.drop('last_review', axis=1)

def cleanAmenities(df):
    df.loc[:, "amenities"] = df.amenities.replace(np.nan, '', regex=True)# fit_transform cannot handle missing values
    df.loc[:, "amenities"] = df.amenities.str.replace(" ", "_").str.replace("-", " ").str.replace("*", "") #split in two because of a python bug (https://stackoverflow.com/questions/3675144/regex-error-nothing-to-repeat)
    vectorizer = CountVectorizer(decode_error = "ignore") 
    X = vectorizer.fit_transform(df.amenities)
    bag_of_words = pd.DataFrame(X.toarray(), columns=vectorizer.get_feature_names())
#    bag_of_words.hist(figsize=(20,20))
    return bag_of_words
#    return pd.concat([df.reset_index(drop=True).drop("amenities", axis = 1), bag_of_words], axis=1)

In [4]:
def cleanAmenities2(df):
    df.drop('n_play', axis=1, inplace=True)
    df.drop('pack_', axis=1, inplace=True)
    
    df.rename(columns={'translation_missing': 'translation_missing_en'}, inplace=True)
    df.drop('_en', axis=1, inplace=True)

    df.rename(columns={'kid_friendly': 'kid_friendly_family'}, inplace=True)
    df.drop('family', axis=1, inplace=True)

    df.rename(columns={'hour_check': '24_hour_check'}, inplace=True)
    df.drop('24', axis=1, inplace=True)

    df.rename(columns={'step': 'step_free_access'}, inplace=True)
    df.drop('free_access', axis=1, inplace=True)

    df.rename(columns={'lit_path_to_entrance': 'well_lit_path_to_entrance'}, inplace=True)
    df.drop('well', axis=1, inplace=True)

    df.rename(columns={'room': 'room_darkening_shades'}, inplace=True)
    df.drop('darkening_shades', axis=1, inplace=True)

    df.rename(columns={'wide_clearance_to_shower': 'wide_clearance_to_shower_toilet'}, inplace=True)
    df.drop('_toilet', axis=1, inplace=True)

    # washer_ is included in washer,  _dryer is already included in _dryer
    df.drop('washer_', axis=1, inplace=True)
    df.drop('_dryer', axis=1, inplace=True)
    return df

In [9]:
jan_path = './united-states/dc/washington-dc/2019-01-17/data/'
jan_listings =  pd.read_csv(jan_path + 'listings.csv.gz')
year = 2019
month = 1
day = 17

df_jan = DropColumns(jan_listings)
df_jan.dropna(subset=['host_since'], inplace=True)
df_jan.dropna(subset=['id'], inplace=True)
df_jan = clean(df_jan, year, month, day)
df_amenities = cleanAmenities(df_jan)
df_amenities = cleanAmenities2(df_amenities)
df_jan = pd.concat([df_jan.reset_index(drop=True).drop("amenities", axis = 1), df_amenities], axis=1)
df_jan.to_csv('listings_2019_01_cleaning.csv',index=False)

/Users/yingli/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3071: DtypeWarning: Columns (61,62) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [10]:
df_jan

,id,host_id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,is_location_exact,...,wheelchair_accessible,wide_clearance_to_bed,wide_clearance_to_shower_toilet,wide_doorway,wide_entryway,wide_hallway_clearance,wifi,window_guards,wine_cooler,wireless_intercom
0,3362,2798,2008-09-07,within an hour,100%,1.0,5.0,1.0,"Shaw, Logan Circle",1,...,0,0,0,0,0,0,1,0,0,0
1,3662,4645,2008-11-26,within an hour,100%,0.0,3.0,1.0,Historic Anacostia,0,...,0,0,0,0,0,0,1,0,0,0
2,3686,4645,2008-11-26,within an hour,100%,0.0,3.0,1.0,Historic Anacostia,0,...,0,0,0,0,0,0,1,0,0,0
3,3771,4795,2008-12-03,unknown,unknown,0.0,1.0,0.0,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",1,...,0,0,0,0,0,0,0,0,0,0
4,4002,5143,2008-12-13,within a few hours,100%,0.0,2.0,1.0,"North Michigan Park, Michigan Park, University...",1,...,0,0,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8989,31628940,54862085,2016-01-16,within a few hours,50-89%,0.0,6.0,1.0,"Downtown, Chinatown, Penn Quarters, Mount Vern...",1,...,0,1,1,3,1,1,1,0,0,0
8990,31629433,54862085,2016-01-16,within a few hours,50-89%,0.0,6.0,1.0,"Downtown, Chinatown, Penn Quarters, Mount Vern...",1,...,0,1,1,3,1,1,1,0,0,0
8991,31635382,234351772,2019-01-04,unknown,unknown,0.0,1.0,0.0,"Colonial Village, Shepherd Park, North Portal ...",1,...,0,0,0,0,0,0,1,0,0,0
8992,31642155,48005494,2015-11-02,within a few hours,100%,1.0,711.0,1.0,"Southwest Employment Area, Southwest/Waterfron...",0,...,0,0,0,0,0,0,1,0,0,0


In [11]:
df_jan.isna().sum().sort_values(ascending=False)

review_scores_value          2019
review_scores_checkin        2017
review_scores_location       2016
review_scores_accuracy       2013
review_scores_cleanliness    2011
                             ... 
well_lit_path_to_entrance       0
laptop_friendly_workspace       0
lake_access                     0
kitchenette                     0
id                              0
Length: 216, dtype: int64

In [12]:
df =  pd.read_csv('listings_2019_01_cleaning.csv')

### Description of each column:
* host_since - date that the host first joined Airbnb
* host_response_time - average amount of time the host takes to reply to messages
* host_response_rate - proportion of messages that the host replies to
* host_acceptance_rate - proportion of the booking requests accepted 
* host_is_superhost - whether or not the host is a superhost, which is a mark of quality for the top-rated and most * * host_listings_count - how many listings the host has in total
* host_identity_verified - whether or not the host has been verified with id
* neighbourhood_cleansed - the London borough the property is in
* property_type - type of property, e.g. house or flat
* room_type - type of listing, e.g. entire home, private room or shared room
* accommodates - how many people the property accommodates
* bathrooms - number of bathrooms  
* bedrooms - number of bedrooms
* beds - number of beds
* bed_type - type of bed, e.g. real bed or sofa-bed
* amenities - list of amenities
* price - nightly advertised price (the target variable)
* security_deposit - the amount required as a security deposit
* cleaning_fee - the amount of the cleaning fee (a fixed amount paid per booking)
* minimum_nights - the minimum length of stay
* maximum_nights - the maximum length of stay
* calendar_updated - when the host last updated the calendar
* availability_30 - how many nights are available to be booked in the next 30 days
* availability_60 - how many nights are available to be booked in the next 60 days
* availability_90 - how many nights are available to be booked in the next 90 days
* availability_365 - how many nights are available to be booked in the next 365 days
* number_of_reviews - the number of reviews left for the property
* number_of_reviews_ltm - the number of reviews left for the property in the last twelve months
* first_review - the date of the first review
* last_review - the date of the most recent review
* review_scores_rating - guests can score properties overall from 1 to 5 stars
* review_scores_accuracy - guests can score the accuracy of a property's description from 1 to 5 stars
* review_scores_cleanliness - guests can score a property's cleanliness from 1 to 5 stars
* review_scores_checkin - guests can score their check-in from 1 to 5 stars
* review_scores_communication - guests can score a host's communication from 1 to 5 stars
* review_scores_location - guests can score a property's location from 1 to 5 stars
* review_scores_value - guests can score a booking's value for money from 1 to 5 stars
* instant_bookable - whether or not the property can be instant booked (i.e. booked straight away, without having to * reviews_per_month - calculated field of the average number of reviews left by guest each month
* cancellation_policy - the type of cancellation policy, e.g. strict or moderate
* reviews_per_month - calculated field of the average number of reviews left by guest each month

In [13]:
for col in df_jan.columns: 
    print(col)

id
host_id
host_since
host_response_time
host_response_rate
host_is_superhost
host_listings_count
host_identity_verified
neighbourhood_cleansed
is_location_exact
property_type
room_type
accommodates
bathrooms
bedrooms
beds
bed_type
price
security_deposit
cleaning_fee
guests_included
extra_people
minimum_nights
maximum_nights
calendar_updated
availability_30
availability_60
availability_90
availability_365
number_of_reviews
first_review
review_scores_rating
review_scores_accuracy
review_scores_cleanliness
review_scores_checkin
review_scores_communication
review_scores_location
review_scores_value
instant_bookable
cancellation_policy
reviews_per_month
host_days_active
time_since_first_review
time_since_last_review
accessible
air_conditioning
amazon_echo
baby_bath
baby_monitor
babysitter_recommendations
balcony
bath_towel
bathroom_essentials
bathtub
bathtub_with_bath_chair
bbq_grill
beach_essentials
bed_linens
bedroom_comforts
bidet
body_soap
breakfast
breakfast_table
building_staff
buzze

In [14]:
df.head()

,id,host_id,host_since,host_response_time,host_response_rate,host_is_superhost,host_listings_count,host_identity_verified,neighbourhood_cleansed,is_location_exact,...,wheelchair_accessible,wide_clearance_to_bed,wide_clearance_to_shower_toilet,wide_doorway,wide_entryway,wide_hallway_clearance,wifi,window_guards,wine_cooler,wireless_intercom
0,3362,2798,2008-09-07,within an hour,100%,1.0,5.0,1.0,"Shaw, Logan Circle",1,...,0,0,0,0,0,0,1,0,0,0
1,3662,4645,2008-11-26,within an hour,100%,0.0,3.0,1.0,Historic Anacostia,0,...,0,0,0,0,0,0,1,0,0,0
2,3686,4645,2008-11-26,within an hour,100%,0.0,3.0,1.0,Historic Anacostia,0,...,0,0,0,0,0,0,1,0,0,0
3,3771,4795,2008-12-03,unknown,unknown,0.0,1.0,0.0,"Columbia Heights, Mt. Pleasant, Pleasant Plain...",1,...,0,0,0,0,0,0,0,0,0,0
4,4002,5143,2008-12-13,within a few hours,100%,0.0,2.0,1.0,"North Michigan Park, Michigan Park, University...",1,...,0,0,0,0,0,0,1,0,0,0


### Merge Table together

In [15]:
#read review statistics data
df_reviews = pd.read_csv('review_stat_byID_byYearMonth.csv')

In [16]:
df_score = df_reviews[df_reviews['YearMonth'] == '2019-1'][['listing_id','score']]

In [17]:
merged_dataframe = pd.merge(left=df_score,right=df, \
                                    how='left', left_on='listing_id', right_on='id')

In [18]:
merged_dataframe.isna().sum()

listing_id                 0
score                      0
id                        25
host_id                   25
host_since                25
                          ..
wide_hallway_clearance    25
wifi                      25
window_guards             25
wine_cooler               25
wireless_intercom         25
Length: 218, dtype: int64